## Digital

This python notebook contains the digital software project, which has an ML model that recognizes the numbers your hands are holding up in real time

In [ ]:
import cv2 as cv
import numpy as np
import time
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe.tasks.python.vision import HandLandmarkerResult

BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

In [4]:
detection_res = None

In [5]:
# Supporting functions

def print_result(result: HandLandmarkerResult, output_image: mp.Image, timestamp_ms: int):
    global detection_res
    detection_res = result

    if result.hand_landmarks:
        print('Hands found!: ', result)

In [6]:
base_opt = python.BaseOptions(model_asset_path='models/hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_opt, num_hands=2, running_mode=mp.tasks.vision.RunningMode.LIVE_STREAM, result_callback=print_result)
detector = vision.HandLandmarker.create_from_options(options)

I0000 00:00:1770613345.477512 11305039 gl_context.cc:407] GL version: 2.1 (2.1 Metal - 90.5), renderer: Apple M4 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1770613345.480572 11305043 inference_feedback_manager.cc:121] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1770613345.483232 11305052 inference_feedback_manager.cc:121] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [ ]:
with HandLandmarker.create_from_options(options) as detector:
    cap = cv.VideoCapture(0)
    start_time = time.time()

    while cap.isOpened():
        success, frame = cap.read()

        if not success:
            print("Can't receive frame :( exiting...")
            break

        timestamp_ms = int((time.time() - start_time)* 1000)

        frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame_rgb)
        
        detector.detect_async(mp_image, timestamp_ms) 

        cv.imshow('frame',frame)

        if cv.waitKey(1) == ord('q'):
            break


I0000 00:00:1770614739.134434 11344587 gl_context.cc:407] GL version: 2.1 (2.1 Metal - 90.5), renderer: Apple M4 Pro
W0000 00:00:1770614739.140262 11344588 inference_feedback_manager.cc:121] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1770614739.148042 11344592 inference_feedback_manager.cc:121] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Hands found!:  HandLandmarkerResult(handedness=[[Category(index=1, score=0.9690747261047363, display_name='Left', category_name='Left')]], hand_landmarks=[[NormalizedLandmark(x=0.716090738773346, y=0.9898681640625, z=-9.206351023749448e-07, visibility=None, presence=None, name=None), NormalizedLandmark(x=0.6731646060943604, y=0.9985331296920776, z=-0.02087034471333027, visibility=None, presence=None, name=None), NormalizedLandmark(x=0.624455988407135, y=0.9721238613128662, z=-0.03696130961179733, visibility=None, presence=None, name=None), NormalizedLandmark(x=0.5987866520881653, y=0.9472000598907471, z=-0.05349302664399147, visibility=None, presence=None, name=None), NormalizedLandmark(x=0.5902147889137268, y=0.9111577272415161, z=-0.06654234975576401, visibility=None, presence=None, name=None), NormalizedLandmark(x=0.6446177959442139, y=0.8310799598693848, z=-0.03538055717945099, visibility=None, presence=None, name=None), NormalizedLandmark(x=0.6343256831169128, y=0.7983494997024536